In [2]:
'''
Code for establishing baseline scores instructor intervention predictions for a portion of the MOOC 
annotation corpus. This code predicts only results for Task 2.1 


'''

'''
Google n gram corpus for idf, --- http://trimc-nlp.blogspot.sg/2013/04/tfidf-with-google-n-grams-and-pos-tags.html
https://news.ycombinator.com/item?id=698991

lexicon for first names -- https://stackoverflow.com/questions/20290870/improving-the-extraction-of-human-names-with-nltk?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
https://nlp.stanford.edu/software/CRF-NER.shtml
https://machinelearningmastery.com/clean-text-machine-learning-python/


add no of posts, comments, no. of posts_comments, no. of sentences, avg. no of sentences per post/comment, 
avg. no of words per post to features
thread length ftrs -- no. posts+comments, posts, comments, no of sentences in thread and no. of words (after removing stop words)

from svm classifer -- output weights of features--word, tfidf weight, svm weight. After classification we can see top 10 weights
add more data
fix cross validation

'''

import numpy as np
import os
import nltk
import pandas as pd
from io import StringIO
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
import operator
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
df = pd.read_csv('../data/21total_preprocessed.csv')
print(tensorflow.VERSION)

/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/radhikanikam/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.4.0


Using TensorFlow backend.


In [3]:
'''
Preparing and printing the data
'''

del df['Task2.1']
print(df.head())
df2 = pd.read_csv('../data/nustotal_preprocessed.csv')
df = df.append(df2)

df['Categories'] = df['Categories'].factorize()[0]
#print((df.head(10)))
print(df['Categories'].value_counts())

   Unnamed: 0                                              Title  \
0           0                        Put the Essays in an e-book   
1           1  Need help with 'common sense' understanding of...   
2           2                            Quiz deadline confusion   
3           3                   HELP--Anyone with good internet?   
4           4  It is not my fault that your site can not with...   

          Course  Categories  \
0  amnhearth-002    resolves   
1  amnhearth-002  elaborates   
2  amnhearth-002    resolves   
3  amnhearth-002    resolves   
4  amnhearth-002    resolves   

                                           Post_text Comment_text  Post_count  \
0  During the recent Human Evolution: Past and Pr...          NaN           1   
1                                                NaN          NaN           0   
2  Today I discovered this course and decided to ...          NaN           3   
3  Hi, I have spent the last three hours trying t...          NaN         

In [4]:
#Removing Names

def extract_entities(text):
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'node'):
                print(chunk.node, ' '.join(c[0] for c in chunk.leaves()))

# from nltk.tag import StanfordNERTagger
# from nltk.tokenize import word_tokenize

# st = StanfordNERTagger('/usr/share/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz',
# 					   '/usr/share/stanford-ner/stanford-ner.jar',
# 					   encoding='utf-8')

# text = 'While in France, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.'

# tokenized_text = word_tokenize(text)
# classified_text = st.tag(tokenized_text)

# print(classified_text)

# nltk.download('words')
# for i in df['Post_text'].dropna():
#     print(i)
#     extract_entities(i)

In [5]:
categories = ['resolves','elaborates','social','requests']
courses = df['Course'].unique()

for course in courses:
    course_data = df.loc[df['Course']==course].dropna()
    post_text = course_data['Post_text'].tolist()
    course_train, course_val, course_test = np.split(course_data.sample(frac=1), [int(.6*len(course_data)), int(.8*len(course_data))])    
df = df.dropna()
#df_train, df_val, df_test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])    
df_train, df_val, y_train, y_val = train_test_split(df.loc[:,'Post_text':'Comment_count'],df['Categories'],test_size=0.40,stratify=df['Categories']) 
#print(df_val)
count_vect = CountVectorizer()
df_train_counts = count_vect.fit_transform(df_train['Post_text'])

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
df_train_tfidf = tfidf_vectorizer.fit(df_train['Post_text'])
text=df_train['Post_text'].tolist()
#print(text[0])
X = df_train_tfidf.transform([text[0]])
#print(tfidf_vectorizer.vocabulary_)
#print(X)
#words_list = list(tfidf_vectorizer.vocabulary_.keys())
#print(words_list)
#print(sorted(tfidf_vectorizer.vocabulary_,key=tfidf_vectorizer.vocabulary_.get, reverse=True))
new_dict={}
#print(tfidf_vectorizer.idf_)
for vocab, idf_score in zip((list(tfidf_vectorizer.vocabulary_.keys())),(tfidf_vectorizer.idf_)):
    #print(vocab,idf_score)
    if len(vocab)>3:
        new_dict[vocab] = idf_score
print(sorted(new_dict.items(),key=operator.itemgetter(1),reverse=True)[:30])

print((X.toarray()))
'''
vectorizer = TfidfVectorizer()

posts=df_train['Post_text'].dropna().tolist()
vectorizer.fit(posts)
#print(vectorizer.vocabulary_)
print(vectorizer.idf_)
vector = vectorizer.transform([posts[0]])
print(vector.shape)
print(vector.toarray())
'''
    

[('question', 4.314186004672526), ('units', 4.314186004672526), ('weights', 4.314186004672526), ('given', 4.314186004672526), ('required', 4.314186004672526), ('answer', 4.314186004672526), ('energy', 4.314186004672526), ('lowest', 4.314186004672526), ('second', 4.314186004672526), ('local', 4.314186004672526), ('minimum', 4.314186004672526), ('possibly', 4.314186004672526), ('points', 4.314186004672526), ('configurations', 4.314186004672526), ('taken', 4.314186004672526), ('account', 4.314186004672526), ('minimums', 4.314186004672526), ('wrote', 4.314186004672526), ('sample', 4.314186004672526), ('octave', 4.314186004672526), ('code', 4.314186004672526), ('solve', 4.314186004672526), ('problem', 4.314186004672526), ('loops', 4.314186004672526), ('calculate', 4.314186004672526), ('remembering', 4.314186004672526), ('case', 4.314186004672526), ('quadratic', 4.314186004672526), ('form', 4.314186004672526), ('expressed', 4.314186004672526)]
[[0. 0. 0. ... 0. 0. 0.]]


/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


"\nvectorizer = TfidfVectorizer()\n\nposts=df_train['Post_text'].dropna().tolist()\nvectorizer.fit(posts)\n#print(vectorizer.vocabulary_)\nprint(vectorizer.idf_)\nvector = vectorizer.transform([posts[0]])\nprint(vector.shape)\nprint(vector.toarray())\n"

In [6]:
# TO DO --  get tf-idf scores from google-ngram-corpus?

# from google_ngram_downloader import readline_google_store
# for word in words_list:
#     count = 0
#     fname, url, records = next(readline_google_store(ngram_len=1, indices=word[0]))
 
#     try:
#         record = next(records)

#         while record.ngram != word:
#             record = next(records)

#         while record.ngram == word:
#             count = count + record.match_count
#             record = next(records)

#     except StopIteration:
#         pass
#     print(count)
    

In [7]:
class_weights = {
    0: 1.,
    1: 1.,
    2: 5.,
    3: 5.
}


# Linear SVM or SGDCLassifier with Hinge Loss
print(" Linear SVM of SGDClassifier with Hinge Loss - Accuracy and feature names for categories")
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
                     ('tfidf', TfidfVectorizer(stop_words='english',ngram_range=(1,2))),
                     ('clf',  SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42 #class_weight=class_weights,
                                            )),
])
text_clf.fit(df_train['Post_text'], y_train)  

predicted = text_clf.predict(df_val['Post_text'])
print("Predicted: "+ str(predicted.tolist()))
print("Actual: " + str(y_val.tolist()))
print("Accuracy: "+ str(np.mean(predicted == y_val))+ "\n")
indices = (np.argsort(text_clf.named_steps['clf'].coef_,axis=1))
#print(indices)
features = (text_clf.named_steps['tfidf'].get_feature_names())
for i in indices:
    print("---"+ str(categories[np.where(np.all(indices==i,axis=1))[0][0]]) + "---")
    for j in i[-15:]:
        print(features[j])
      
 


 Linear SVM of SGDClassifier with Hinge Loss - Accuracy and feature names for categories
Predicted: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 2, 0, 2, 0, 1, 0, 0]
Actual: [2, 3, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 1, 1, 1]
Accuracy: 0.5405405405405406

---resolves---
kc
ball
bits
ft
explanation
velocity
noise
address
miss
zero
page
bars
force
acceleration
amp
---elaborates---
concentration
probability
average
film
artist
andy
negative
screen tests
tests
screen
distribution
conceptual
rate
art
warhol
---social---
instructions
coursera
free course
vec
2012 001
date
correct answer
urlref 2012
2012
urlref4
videos
14
dates
pmatrix
course
---requests---
zeger
solved
contemporary
learn
duchamp
learn doing
koons damien
like jeff
head
damien hirst
damien
jeff koons
hirst
jeff
koons


/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [8]:
print('----------Linear SVM or SGDCLassifier with Hinge Loss------------\n\n')
print(metrics.confusion_matrix(y_val, predicted))
print(metrics.classification_report(y_val, predicted))

# scores = cross_val_score(
#     text_clf, X, df_val['Categories'], cv=2, scoring='f1_macro')

----------Linear SVM or SGDCLassifier with Hinge Loss------------


[[15  4  0  0]
 [ 9  3  0  0]
 [ 2  0  2  0]
 [ 1  1  0  0]]
             precision    recall  f1-score   support

          0       0.56      0.79      0.65        19
          1       0.38      0.25      0.30        12
          2       1.00      0.50      0.67         4
          3       0.00      0.00      0.00         2

avg / total       0.52      0.54      0.50        37



/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
# Gradient Boosted Trees

class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document """

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text)
                }
                for text in posts]

class DenseTransformer(TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

text_clf = Pipeline([
                     ('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
                     
                     ('to_dense', DenseTransformer()),
                     
                     ('clf', GradientBoostingClassifier(random_state=42
                                            )),
])


print('Gradient Boosted Trees -Accuracy and Most Important Features')

text_clf.fit(df_train['Post_text'], y_train,)  

predicted = text_clf.predict(df_val['Post_text'])
#print(df_val['Post_text'],y_val)
print("Predicted: "+ str(predicted.tolist()))
print("Actual: " + str(y_val.tolist()))
print("Accuracy: "+ str(np.mean(predicted == y_val))+ "\n")


#print(np.sort(text_clf.named_steps['clf'].feature_importances_))
indices = np.argsort(text_clf.named_steps['clf'].feature_importances_)[-15:]


features = (text_clf.named_steps['tfidf'].get_feature_names())
print("\nMost important features: \n")
for i in indices:
    print(features[i])
# important_features_dict = {}
# for x,i in enumerate(text_clf.named_steps['clf'].feature_importances_):
#     important_features_dict[x]=i


# important_features_list = sorted(important_features_dict,
#                                  key=important_features_dict.get,
#                                  reverse=True)

# print('Most important features: %s' %important_features_list)

Gradient Boosted Trees -Accuracy and Most Important Features


/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Predicted: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 2, 1, 2, 0, 1, 0, 1]
Actual: [2, 3, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 1, 1, 1]
Accuracy: 0.5405405405405406


Most important features: 

math
consider
wondering
provided
ll
gave
answers
similar
urlref urlref
like
trying figure
quiz
rate
idea
videos


In [10]:
print('----------Gradient Boosted Trees------------\n\n')
print(metrics.confusion_matrix(y_val, predicted))
print(metrics.classification_report(y_val, predicted))



----------Gradient Boosted Trees------------


[[14  3  2  0]
 [ 8  4  0  0]
 [ 2  0  2  0]
 [ 1  1  0  0]]
             precision    recall  f1-score   support

          0       0.56      0.74      0.64        19
          1       0.50      0.33      0.40        12
          2       0.50      0.50      0.50         4
          3       0.00      0.00      0.00         2

avg / total       0.50      0.54      0.51        37



/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
# Random Forest Pipeline

text_clf = Pipeline([
                     ('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
                     
                     ('to_dense', DenseTransformer()),
                     
                     ('clf', RandomForestClassifier(class_weight=class_weights,random_state=42
                                            )),
])


print('Random Forest Pipeline - Accuracy and Most Important Features ')

text_clf.fit(df_train['Post_text'], y_train)  

predicted = text_clf.predict(df_val['Post_text'])
#print(df_val['Post_text'],y_val)
print("Predicted: "+ str(predicted.tolist()))
print("Actual: " + str(y_val.tolist()))
print("Accuracy: "+ str(np.mean(predicted == y_val))+ "\n")

#print(np.sort(text_clf.named_steps['clf'].feature_importances_))
indices = np.argsort(text_clf.named_steps['clf'].feature_importances_)[-15:]
#print(indices)

features = (text_clf.named_steps['tfidf'].get_feature_names())
print("\nMost important features: \n")
for i in indices:
    print(features[i])


Random Forest Pipeline - Accuracy and Most Important Features 
Predicted: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
Actual: [2, 3, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 1, 1, 1]
Accuracy: 0.5945945945945946



/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):



Most important features: 

culture
developing type
majored biology
round second
thread thread_id
idea
released weeks
similar
understand concepts
sculptures
instructions didn
pdf week
video
integral
math hope


In [12]:
print('----------Random Forest------------\n\n')
print(metrics.confusion_matrix(y_val, predicted))
print(metrics.classification_report(y_val, predicted))

----------Random Forest------------


[[19  0  0  0]
 [ 9  3  0  0]
 [ 4  0  0  0]
 [ 2  0  0  0]]
             precision    recall  f1-score   support

          0       0.56      1.00      0.72        19
          1       1.00      0.25      0.40        12
          2       0.00      0.00      0.00         4
          3       0.00      0.00      0.00         2

avg / total       0.61      0.59      0.50        37



/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
'''
Download and Import glove 
'''
print('Download and import glove word embeddings to get the word vectors')

BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))
        

Download and import glove word embeddings to get the word vectors
Found 400000 word vectors.


In [14]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2

In [15]:
import re
df = pd.read_csv('../data/21total_preprocessed.csv').dropna(subset=['Post_text'])
del df['Task2.1']
df['Categories'] = df['Categories'].factorize()[0]
texts = df['Post_text']
print((texts[0][0]))
labels = df['Categories']

D


In [17]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)

tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
#print(word_index)

Found 7016 unique tokens.


In [18]:
print(len(texts))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

315
Shape of data tensor: (315, 1000)
Shape of label tensor: (315, 4)


In [19]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
print(labels)
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 ...
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [20]:
label1 = []
for entry in labels:
    label1.append(np.argmax(entry))
print(np.array(label1).shape)
label1 =np.array(label1)
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Preparing embedding matrix.')
print(y_train.shape)

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

        

(315,)
Preparing embedding matrix.
(252, 4)


In [21]:
import keras.backend as K

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print('Training model.')

from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from keras.layers import Flatten


# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)

#x = Dense(128,activation='relu')(embedded_sequences)
#x = Flatten()(x)
preds = Dense(4, activation='softmax')(x)

model = Model(sequence_input, preds)
class Metrics(Callback):
    
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
 
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict,average=None)
        _val_recall = recall_score(val_targ, val_predict,average=None)
        _val_precision = precision_score(val_targ, val_predict,average=None)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        return 
    

        
        
metrics1 = Metrics()


model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=12,
          epochs=10,
          validation_data=(x_val, y_val),callbacks=[metrics1])

print((metrics1.val_f1s[-1], metrics1.val_precisions[-1], metrics1.val_recalls[-1]))

Training model.
Train on 252 samples, validate on 63 samples
Epoch 1/10
252/252 [==============================] - 9s 35ms/step - loss: 1.2725 - acc: 0.4960 - val_loss: 1.0369 - val_acc: 0.5873

/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/radhikanikam/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 2/10
252/252 [==============================] - 9s 34ms/step - loss: 1.0242 - acc: 0.5833 - val_loss: 1.0004 - val_acc: 0.6032

In [22]:
class_weights = {
    0: 1.,
    1: 1.,
    2: 5.,
    3: 5.
}

label1 = []
for entry in labels:
    label1.append(np.argmax(entry))
print(np.array(label1).shape)
label1 =np.array(label1)
x_train = data[:-num_validation_samples]
y_train = label1[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = label1[-num_validation_samples:]

#Linear SVM with Glove 
print(embedding_matrix.shape)
from sklearn import linear_model
clf = linear_model.SGDClassifier(class_weight=class_weights)

#labels = np.array(labels)
#print(y_train.shape)
clf.fit(x_train,y_train)

predicted = clf.predict(x_val)
print("Predicted: "+ str(predicted.tolist()))
print("Actual: " + str(y_val.tolist()))
print(len(y_val.tolist()))
print("Accuracy: "+ str(np.mean(predicted == y_val))+ "\n")
#indices = (np.argsort(clf.named_steps['clf'].coef_,axis=1))
#print(indices)
#features = (clf.named_steps['tfidf'].get_feature_names())
# for i in indices:
#     print("---"+ str(categories[np.where(np.all(indices==i,axis=1))[0][0]]) + "---")
#     for j in i[-15:]:
#         print(features[j])


(315,)
(7017, 300)
Predicted: [2, 2, 2, 0, 2, 2, 0, 0, 0, 2, 2, 3, 0, 0, 0, 1, 0, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 0, 2, 1, 1, 2, 3, 0, 2, 2, 1, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2]
Actual: [0, 2, 0, 0, 0, 0, 2, 0, 1, 0, 1, 2, 0, 2, 0, 2, 0, 1, 2, 0, 2, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 1, 0, 1, 0, 2, 2, 0, 0, 3, 0, 0, 2, 0, 0, 0, 0, 1, 2, 0, 1, 2, 2, 0, 0, 0, 3, 2, 0, 0, 2, 0, 0]
63
Accuracy: 0.30158730158730157



In [23]:
print(metrics.confusion_matrix(y_val, predicted))
print(metrics.classification_report(y_val, predicted))



[[ 8  7 22  0]
 [ 1  3  3  1]
 [ 4  2  8  1]
 [ 2  0  1  0]]
             precision    recall  f1-score   support

          0       0.53      0.22      0.31        37
          1       0.25      0.38      0.30         8
          2       0.24      0.53      0.33        15
          3       0.00      0.00      0.00         3

avg / total       0.40      0.30      0.30        63

